# Experiments with Home Credit

In [2]:
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score, brier_score_loss, roc_curve
from sklearn.inspection import PartialDependenceDisplay
from lime import lime_tabular
import shap
from cfmining.algorithms import MAPOCAM, BruteForce, Greedy
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import MonotoneClassifier
from cfmining.visualization import buildTable, PlotCounterfactuals
from cfmining.mip_builder import RecourseBuilder
from cfmining.action_set import ActionSet
import joblib


import credit_pipeline.data_exploration as dex
from credit_pipeline.training import *


%load_ext autoreload
%autoreload 2

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## Loading and Cleaning

In [3]:
seed_number = 0

In [4]:
path = "../data/HomeCredit/"
df = dex.read_csv_encoded(path, 'application_train.csv')

In [5]:
columns_to_drop = dex.check_missing(df, 50,  False)
columns_to_drop

['COMMONAREA_MEDI',
 'COMMONAREA_AVG',
 'COMMONAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAPARTMENTS_MEDI',
 'FONDKAPREMONT_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAPARTMENTS_MEDI',
 'FLOORSMIN_AVG',
 'FLOORSMIN_MODE',
 'FLOORSMIN_MEDI',
 'YEARS_BUILD_MEDI',
 'YEARS_BUILD_MODE',
 'YEARS_BUILD_AVG',
 'OWN_CAR_AGE',
 'LANDAREA_MEDI',
 'LANDAREA_MODE',
 'LANDAREA_AVG',
 'BASEMENTAREA_MEDI',
 'BASEMENTAREA_AVG',
 'BASEMENTAREA_MODE',
 'EXT_SOURCE_1',
 'NONLIVINGAREA_MODE',
 'NONLIVINGAREA_AVG',
 'NONLIVINGAREA_MEDI',
 'ELEVATORS_MEDI',
 'ELEVATORS_AVG',
 'ELEVATORS_MODE',
 'WALLSMATERIAL_MODE',
 'APARTMENTS_MEDI',
 'APARTMENTS_AVG',
 'APARTMENTS_MODE',
 'ENTRANCES_MEDI',
 'ENTRANCES_AVG',
 'ENTRANCES_MODE',
 'LIVINGAREA_AVG',
 'LIVINGAREA_MODE',
 'LIVINGAREA_MEDI',
 'HOUSETYPE_MODE']

In [6]:
df = df.drop(columns_to_drop, axis=1)

In [7]:
df_cols = df.columns.to_list()
obj_cols = dex.list_by_type(df, ['O'])

In [8]:
def days_to_years(dataframe, col_name):
        """
        Converts values from string to numeric.
        Uses the map function to convert the information on days employed to years employed
        """
        df_name = dataframe.copy()

        if col_name in df.columns:
            #Converts values from string to numeric.
            df_name[col_name] = pd.to_numeric(df_name[col_name], errors='coerce')

            #drops null values on the column
            df_name = df_name.dropna(subset=[col_name])

            #Use the map function to convert the information on days employed to years employed
            year = df_name.loc[:, col_name].map(lambda x: int(abs(x / 365)), na_action=None)
            df_name['YEARS'+col_name[4:]] = year

            #drops the column
            df_name = df_name.drop(col_name, axis=1)

        return df_name


df = days_to_years(df, "DAYS_EMPLOYED")
df = days_to_years(df, 'DAYS_BIRTH')

## Training Basic Models

In [9]:
X_acp = df.iloc[:, (df.columns != "TARGET") & (df.columns != "SK_ID_CURR")]
y_acp = df["TARGET"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X_acp, 
    y_acp, 
    test_size = 0.2, 
    random_state=seed_number, 
    stratify=y_acp
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, 
    y_train, 
    test_size = 0.2, 
    random_state=seed_number, 
    stratify=y_train
)

In [15]:
classifiers = {
    "Logistic Regression": LogisticRegression,
    "Random Forest": RandomForestClassifier,
    "LightGBM": LGBMClassifier,
    "MLPC" : MLPClassifier,
}

In [32]:
param_spaces = {
    "LogisticRegression": {
        'C': {'low': 0.001, 'high': 10, 'log': True, 'type':'float'},
        'max_iter': {'low': 1000, 'high': 1000, 'step':1, 'type':'int'},
        'penalty': {'choices': ['l2'], 'type':'categorical'}
    },
    "RandomForestClassifier": {
        'n_estimators': {'low':10, 'high':150, 'step':20, 'type':'int'},
        'max_depth': {'low':2, 'high':10, 'type':'int'},
        'criterion': {'choices':['gini', 'entropy'], 'type':'categorical'},
        'min_samples_leaf' : {"low" : 1, "high" : 51, "step" : 5, 'type':'int'},
        "max_features" : {"low" : 0.1, "high" : 1.0, "type" : "float"},
    },
    "LGBMClassifier": {
        'learning_rate': {'low': 0.01, 'high': 1.0, 'type': 'float', 'log': True},
        "num_leaves" : {"low" : 10, "high" : 100, "step" : 5, 'type':'int'},
        'max_depth': {'low': 2, 'high': 10, 'type': 'int'},
        'min_child_samples': {'low': 1, 'high': 51, 'step': 5, 'type': 'int'},
        'colsample_bytree': {'low': 0.1, 'high': 1.0, 'type': 'float'},
        'reg_alpha': {'low': 0.0, 'high': 1.0, 'type': 'float'},
        'reg_lambda': {'low': 0.0, 'high': 1.0, 'type': 'float'},
        'n_estimators': {'low': 10, 'high': 100, 'step': 10, 'type': 'int'},
        "verbose" : {"choices" : [-1], 'type':'categorical'}
    },
    "MLPClassifier": {
        "hidden_layer_sizes" : {"choices" : [
            [128, 64, 32],
            [128, 64, 32, 16],
            [256, 128, 64, 32, 16],
        ], 'type':'categorical'},
        "alpha" : {'low': 0.0001, 'high': 0.01, 'type': 'float', 'log': True},
        "learning_rate" : {'choices': ['constant', 'invscaling', 'adaptive'], 'type':'categorical'},
        "learning_rate_init" : {'low': 0.001, 'high': 0.1, 'type': 'float', 'log': True},
        "early_stopping" : {'choices': [True], 'type':'categorical'},
        "max_iter" : {"choices" : [50], 'type':'categorical'},
    }
}

In [ ]:
study_logistic, model_logistic = optimize_model(LogisticRegression, param_spaces["LogisticRegression"], X_train, y_train, X_val , y_val, n_trials=100)
joblib.dump(model_logistic, "models/model_logistic.joblib")

In [13]:
print("Score for Logistic Regression: ", study_logistic.best_value)
print(study_logistic.best_params)

Score for Logistic Regression:  0.7416166537417906
{'C': 9.891796347509901, 'max_iter': 1000, 'penalty': 'l2'}


In [ ]:
study_rf, model_rf = optimize_model(RandomForestClassifier, param_spaces["RandomForestClassifier"], X_train, y_train, X_val , y_val, n_trials=100)
joblib.dump(model_rf, "models/model_rf.joblib")

In [15]:
print("Score for Random Forest: ", study_rf.best_value)
print(study_rf.best_params)

Score for Random Forest:  0.7413317275761191
{'n_estimators': 130, 'max_depth': 10, 'criterion': 'gini', 'min_samples_leaf': 26, 'max_features': 0.22758887452086007}


In [ ]:
study_lgbm, model_lgbm = optimize_model(LGBMClassifier, param_spaces["LGBMClassifier"], X_train, y_train, X_val , y_val, n_trials=100)
joblib.dump(model_lgbm, "models/model_lgbm.joblib")

In [17]:
print("Score for LGBM: ", study_lgbm.best_value)
print(study_lgbm.best_params)

Score for LGBM:  0.7540305519133603
{'learning_rate': 0.18595982860752458, 'num_leaves': 100, 'max_depth': 5, 'min_child_samples': 36, 'colsample_bytree': 0.3863921178517447, 'reg_alpha': 0.9785871345777682, 'reg_lambda': 0.586360544180212, 'n_estimators': 100, 'verbose': -1}


In [33]:
study_mlp, model_mlp = optimize_model(MLPClassifier, param_spaces["MLPClassifier"], X_train, y_train, X_val , y_val, n_trials=100)
joblib.dump(model_mlp, "models/model_mlp.joblib")

[I 2023-09-29 11:14:45,278] A new study created in memory with name: no-name-5359287d-b6ea-4f77-b62b-f4d68f1f42bd


/home/giovani/anaconda3/envs/credit_pipeline/lib/python3.8/site-packages/optuna/distributions.py:524: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 64, 32] which is of type list.

/home/giovani/anaconda3/envs/credit_pipeline/lib/python3.8/site-packages/optuna/distributions.py:524: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 64, 32, 16] which is of type list.

/home/giovani/anaconda3/envs/credit_pipeline/lib/python3.8/site-packages/optuna/distributions.py:524: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [256, 128, 64, 32, 16] which is of type list.

[I 2023-09-29 11:15:18,955] Trial 0 finished with value: 0.7259999273045299 and parameters: {'hidden_layer_sizes': [128, 64, 32, 16], 'alpha': 0

['models/model_mlp.joblib']

In [34]:
print("Score for MLP: ", study_mlp.best_value)
print(study_mlp.best_params)

Score for MLP:  0.7433528703800805
{'hidden_layer_sizes': [128, 64, 32], 'alpha': 0.0003527051808306031, 'learning_rate': 'invscaling', 'learning_rate_init': 0.010035211915818264, 'early_stopping': True, 'max_iter': 50}


## Model evaluation

In [35]:
model_logistic = joblib.load("models/model_logistic.joblib")
model_rf = joblib.load("models/model_rf.joblib")
model_lgbm = joblib.load("models/model_lgbm.joblib")
model_mlp = joblib.load("models/model_mlp.joblib")

In [19]:
def get_opt_pred(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    opt_threshold = thresholds[np.argmax(tpr - fpr)]
    y_pred = (y_score >= opt_threshold).astype('int')
    return y_pred

def get_metrics(y_true, y_score):
    y_pred = get_opt_pred(y_true, y_score)
    
    metrics_dict = {
        "roc_auc" : roc_auc_score(y_true, y_score),
        "balanced_accuracy" : balanced_accuracy_score(y_true, y_pred),
        "accuracy" : accuracy_score(y_true, y_pred),
        "precision" : precision_score(y_true, y_pred, zero_division=0),
        "recall" : recall_score(y_true, y_pred),
        "f1" : f1_score(y_true, y_pred),
        "brier_score" : brier_score_loss(y_true, y_score),
    }
    return metrics_dict

def get_fairness_metrics(y_true, y_score, sensitive_attr):
    # sensitive attribute must be 0 or 1
    y_pred = get_opt_pred(y_true, y_score)
    # statistical disparity
    p_y1_z1 = np.mean(y_pred[sensitive_attr == 1])
    p_y1_z0 = np.mean(y_pred[sensitive_attr == 0])
    spd = p_y1_z1 - p_y1_z0

    # equalized odds
    tpr_z1 = recall_score(y_true[sensitive_attr == 1], y_pred[sensitive_attr == 1])
    tpr_z0 = recall_score(y_true[sensitive_attr == 0], y_pred[sensitive_attr == 0])
    eod = tpr_z1 - tpr_z0

    # geometric mean of accuracies
    accuracy_z1 = accuracy_score(y_true[sensitive_attr == 1], y_pred[sensitive_attr == 1])
    accuracy_z0 = accuracy_score(y_true[sensitive_attr == 0], y_pred[sensitive_attr == 0])
    gma = np.sqrt(accuracy_z1 * accuracy_z0)

    metrics_dict = {
        "statistical_disparity" : spd,
        "equalized_odds" : eod,
        "geometric_mean_accuracy" : gma,
    }
    return metrics_dict


In [36]:
sensitive = X_test.CODE_GENDER == "F"

results = []
metrics_dict = {"model" : "LogisticRegression"}
metrics_dict.update(get_metrics(y_test, model_logistic.predict_proba(X_test)[:, 1]))
metrics_dict.update(get_fairness_metrics(y_test, model_logistic.predict_proba(X_test)[:, 1], sensitive))
results.append(metrics_dict)

metrics_dict = {"model" : "RandomForest"}
metrics_dict.update(get_metrics(y_test, model_rf.predict_proba(X_test)[:, 1]))
metrics_dict.update(get_fairness_metrics(y_test, model_rf.predict_proba(X_test)[:, 1], sensitive))
results.append(metrics_dict)

metrics_dict = {"model" : "LGBM"}
metrics_dict.update(get_metrics(y_test, model_lgbm.predict_proba(X_test)[:, 1]))
metrics_dict.update(get_fairness_metrics(y_test, model_lgbm.predict_proba(X_test)[:, 1], sensitive))
results.append(metrics_dict)

metrics_dict = {"model" : "MLP"}
metrics_dict.update(get_metrics(y_test, model_mlp.predict_proba(X_test)[:, 1]))
metrics_dict.update(get_fairness_metrics(y_test, model_mlp.predict_proba(X_test)[:, 1], sensitive))
results.append(metrics_dict)

pd.DataFrame(results)

,model,roc_auc,balanced_accuracy,accuracy,precision,recall,f1,brier_score,statistical_disparity,equalized_odds,geometric_mean_accuracy
0,LogisticRegression,0.749436,0.686794,0.649627,0.152240,0.731118,0.252005,0.068458,-0.176512,-0.147892,0.624284
1,RandomForest,0.742781,0.684336,0.688682,0.161139,0.679154,0.260477,0.068967,-0.187267,-0.166408,0.661380
2,LGBM,0.756379,0.694077,0.671967,0.159945,0.720443,0.261773,0.067977,-0.171929,-0.141499,0.647361
3,MLP,0.738606,0.677440,0.656748,0.150792,0.702115,0.248264,0.069195,-0.156405,-0.136732,0.634897


## Explainability

**TODO**

*   Add easy way to include default parameters in optuna
*   Warnings on pipeline given numeric unique number of values
